In [1]:
import boto3
import pandas as pd
from io import StringIO,BytesIO
from datetime import datetime,timedelta

# Adapter Layer

In [2]:
def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
    csv_obj=bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data=StringIO(csv_obj)
    df=pd.read_csv(data,delimiter=sep)
    return df

def write_df_to_s3(bucket,df,key):
    out_buffer=BytesIO()
    df.to_parquet(out_buffer,index=False)
    bucket.put_object(Body=out_buffer.getvalue(),Key=key)
    return True

def list_files_in_prefix(bucket,prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files


# Application Layer

In [3]:
def extract(bucket,date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket,date)]
    df=pd.concat([read_csv_to_df(bucket,obj) for obj in files],ignore_index=True)
    return df

def transform_report1(df,columns,arg_date):
    df=df.loc[:,columns]
    df.dropna(inplace=True)
    df['opening_price']=df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price']=df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df=df.groupby(['ISIN','Date'],as_index=False).agg(opening_price_eur=('opening_price','min'),
                                                         closing_price_eur=('closing_price','min'),
                                                         minimum_price_eur=('MinPrice','min'),
                                                         maximum_price_eur=('MaxPrice','max'),
                                                         daily_traded_volumne=('TradedVolume','sum'))
    df['prev_closing_price']=df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%']=(df['closing_price_eur']-df['prev_closing_price'])/df['prev_closing_price']*100
    df.drop(columns=['prev_closing_price'],inplace=True)
    df=df.round(decimals=2)
    df=df[df.Date>=arg_date]
    return df

def load(bucket,df,trg_key,trg_format):
    key=trg_key+datetime.today().strftime('%Y%m%d_%H%M%S')+trg_format
    write_df_to_s3(bucket,df,key)
    return True

def etl_report1(src_bucket,trg_bucket,date_list,columns,arg_date,trg_key,trg_format):
    df=extract(src_bucket,date_list)
    df=transform_report1(df,columns,arg_date)
    load(trg_bucket,df,trg_key,trg_format)
    return True

def return_date_list(bucket,arg_date,src_format):
    min_date=datetime.strptime(arg_date,src_format).date()-timedelta(days=1)
    today=datetime.today().date()
    return_date_list = [(min_date+timedelta(days=x)).strftime(src_format) for x in range(0,(today-min_date).days+1)]
    return return_date_list

In [4]:
# main fuction entrypoint
def main():
    trg_key='xetra_daily_report_'
    trg_format='.parquet'
    arg_date='2022-01-20'
    src_format='%Y-%m-%d'
    src_bucket='deutsche-boerse-xetra-pds'
    trg_bucket='YOUR_TRG_BUCKET'
    columns=['ISIN','Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume']
    
    # init
    s3 = boto3.resource('s3',aws_access_key_id="AWS_ACCESS_KEY",
                         aws_secret_access_key="AWS_SCERET_KEY")
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    date_list = return_date_list(bucket_src ,arg_date ,src_format)
    etl_report1(bucket_src ,bucket_trg,date_list ,columns ,arg_date ,trg_key ,trg_format)



main()

# Reading the uploaded file

In [5]:
trg_bucket='YOUR_TRG_BUCKET'
s3 = boto3.resource('s3',aws_access_key_id="AWS_ACCESS_KEY",
                         aws_secret_access_key="AWS_SCERET_KEY")
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20220120_194643.parquet


In [7]:
prq_obj=bucket_trg.Object(key="xetra_daily_report_20220120_194643.parquet").get().get('Body').read()
data=BytesIO(prq_obj)
df_report=pd.read_parquet(data)

df_report.head()

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volumne,change_prev_closing_%
0,AT000000STR1,2022-01-20,38.20,38.20,38.20,38.20,100,0.66
1,AT0000606306,2022-01-20,25.04,24.60,24.52,25.42,12103,-3.00
2,AT0000644505,2022-01-20,118.00,116.80,116.40,118.80,279,-2.18
3,AT0000652011,2022-01-20,43.00,43.25,43.00,43.72,705,-0.07
4,AT0000652250,2022-01-20,22.00,22.00,22.00,22.00,32,-0.45
